# finetune pipeline test

## test1:lora+pearson_delta

### 1. finetune

In [ ]:
export PYTHONPATH=/data3/fanpeishan/state:$PYTHONPATH
python for_state/finetune_pipeline/scripts/run_finetune.py \
  --config for_state/finetune_pipeline/examples/lora_pearson.yaml

### 2. preprocess infer

In [ ]:
Saving preprocessed data to /data3/fanpeishan/state/for_state/run_results/run14/c37_prep_preprocess.h5ad

### 3. infer

In [ ]:
export CUDA_VISIBLE_DEVICES=4
state tx infer \
  --model-dir /data3/fanpeishan/state/for_state/models/ST-Tahoe \
  --checkpoint /data3/fanpeishan/state/for_state/models/ST-Tahoe/final_from_preprint.ckpt \
  --pert-col 'drugname_drugconc' \
  --control-pert "[('DMSO_TF', 0.0, 'uM')]" \
  --celltype-col cell_name \
  --batch-col 'plate' \
  --adata /data3/fanpeishan/state/for_state/run_results/run14/c37_prep_preprocess.h5ad \
  --output /data3/fanpeishan/state/for_state/finetune_pipeline/results/test2/c37_prep_preprocess_infer.h5ad \
  --quiet \
  --lora-path /data3/fanpeishan/state/for_state/finetune_pipeline/results/test2/checkpoint_epoch10.pt 

In [ ]:
StateTransitionPerturbationModel(                                                                                                           [28/1047]
  (loss_fn): SamplesLoss()                                                                                                                           
  (pert_encoder): Sequential(                                                                                                                        
    (0): Linear(in_features=1138, out_features=1488, bias=True)                                                                                      
    (1): GELU(approximate='none')                                                                                                                    
    (2): Dropout(p=0.1, inplace=False)                                                                                                               
    (3): Linear(in_features=1488, out_features=1488, bias=True)                                                                                      
    (4): GELU(approximate='none')                                                                                                                    
    (5): Dropout(p=0.1, inplace=False)                                                                                                               
    (6): Linear(in_features=1488, out_features=1488, bias=True)                                                                                      
    (7): GELU(approximate='none')                                                                                                                    
    (8): Dropout(p=0.1, inplace=False)                                                                                                               
    (9): Linear(in_features=1488, out_features=1488, bias=True)                                                                                      
  )                                                                                                                                                  
  (basal_encoder): Sequential(                                                                                                                       
    (0): Linear(in_features=2000, out_features=1488, bias=True)                                                                                      
    (1): GELU(approximate='none')                                                                                                                    
    (2): Dropout(p=0.1, inplace=False)                                                                                                               
    (3): Linear(in_features=1488, out_features=1488, bias=True)                                                                                      
    (4): GELU(approximate='none')                                                                                                                    
    (5): Dropout(p=0.1, inplace=False)                                                                                                               
    (6): Linear(in_features=1488, out_features=1488, bias=True)                                                                                      
    (7): GELU(approximate='none')                                                                                                                    
    (8): Dropout(p=0.1, inplace=False)                                                                                                               
    (9): Linear(in_features=1488, out_features=1488, bias=True)                                                                                      
  )                                                                                                                                                  
  (transformer_backbone): LlamaModel(                                                                                                                
    (embed_tokens): Embedding(32000, 1488, padding_idx=0)                                                                                            
    (layers): ModuleList(                                                                                                                            
      (0-5): 6 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=1488, out_features=1488, bias=False)
          (k_proj): Linear(in_features=1488, out_features=1488, bias=False)
          (v_proj): Linear(in_features=1488, out_features=1488, bias=False)
          (o_proj): Linear(in_features=1488, out_features=1488, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=1488, out_features=5952, bias=False)
          (up_proj): Linear(in_features=1488, out_features=5952, bias=False)
          (down_proj): Linear(in_features=5952, out_features=1488, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((1488,), eps=1e-06)
        (post_attention_layernorm): LlamaRMSNorm((1488,), eps=1e-06)
      )
    )
    (norm): LlamaRMSNorm((1488,), eps=1e-06)
    (rotary_emb): LlamaRotaryEmbedding()
  )
  (project_out): Sequential(
    (0): Linear(in_features=1488, out_features=1488, bias=True)
    (1): GELU(approximate='none')
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=1488, out_features=1488, bias=True)
    (4): GELU(approximate='none')
    (5): Dropout(p=0.1, inplace=False)
    (6): Linear(in_features=1488, out_features=1488, bias=True)
    (7): GELU(approximate='none')
    (8): Dropout(p=0.1, inplace=False)
    (9): Linear(in_features=1488, out_features=2000, bias=True)
  )
  (relu): ReLU()
)

=== Inference complete ===
Input cells:         1835947
Controls simulated:  45150
Treated simulated:   1790797
Wrote predictions to adata.X
Saved:               /data3/fanpeishan/state/for_state/finetune_pipeline/results/test2/c37_prep_preprocess_infer.h5ad

### 4. eval

In [ ]:
cell-eval run \
    -ap /data3/fanpeishan/state/for_state/finetune_pipeline/results/test2/c37_prep_preprocess_infer.h5ad \
    -ar /data3/fanpeishan/state/for_state/run_results/run14/c37_prep.h5ad \
    -o /data3/fanpeishan/state/for_state/finetune_pipeline/results/test2 \
    --control-pert "[('DMSO_TF', 0.0, 'uM')]" \
    --pert-col 'drugname_drugconc' \
    --profile full \
    --batch-size 1024 \
    --num-threads 10 

In [4]:
import pandas as pd
results=pd.read_csv('/data3/fanpeishan/state/for_state/finetune_pipeline/results/test2/agg_results.csv')
mean_results=results[results.statistic == 'mean'][['overlap_at_100','pearson_delta','mse','overlap_at_N']]
print(mean_results)

   overlap_at_100  pearson_delta       mse  overlap_at_N
2        0.096038       0.357741  0.024972      0.127149


## test2:lora+all_combined_loss

### 1. finetune

In [ ]:
export PYTHONPATH=/data3/fanpeishan/state:$PYTHONPATH
python for_state/finetune_pipeline/scripts/run_finetune.py \
  --config for_state/finetune_pipeline/examples/lora_multi_loss.yaml

### 2. preprocess infer

In [ ]:
Saving preprocessed data to /data3/fanpeishan/state/for_state/run_results/run14/c37_prep_preprocess.h5ad

### 3. infer

In [ ]:
export CUDA_VISIBLE_DEVICES=6
state tx infer \
  --model-dir /data3/fanpeishan/state/for_state/models/ST-Tahoe \
  --checkpoint /data3/fanpeishan/state/for_state/models/ST-Tahoe/final_from_preprint.ckpt \
  --pert-col 'drugname_drugconc' \
  --control-pert "[('DMSO_TF', 0.0, 'uM')]" \
  --celltype-col cell_name \
  --batch-col 'plate' \
  --adata /data3/fanpeishan/state/for_state/run_results/run14/c37_prep_preprocess.h5ad \
  --output /data3/fanpeishan/state/for_state/finetune_pipeline/results/lora_multi_loss/c37_prep_preprocess_infer.h5ad \
  --quiet \
  --lora-path /data3/fanpeishan/state/for_state/finetune_pipeline/results/lora_multi_loss/checkpoint_epoch2.pt 

### 4. eval

In [ ]:
cell-eval run \
    -ap /data3/fanpeishan/state/for_state/finetune_pipeline/results/lora_multi_loss/c37_prep_preprocess_infer.h5ad \
    -ar /data3/fanpeishan/state/for_state/run_results/run14/c37_prep.h5ad \
    -o /data3/fanpeishan/state/for_state/finetune_pipeline/results/lora_multi_loss \
    --control-pert "[('DMSO_TF', 0.0, 'uM')]" \
    --pert-col 'drugname_drugconc' \
    --profile full \
    --batch-size 1024 \
    --num-threads 10 

In [5]:
import pandas as pd
results=pd.read_csv('/data3/fanpeishan/state/for_state/finetune_pipeline/results/lora_multi_loss/agg_results.csv')
mean_results=results[results.statistic == 'mean'][['overlap_at_100','pearson_delta','mse','overlap_at_N']]
print(mean_results)

   overlap_at_100  pearson_delta       mse  overlap_at_N
2        0.083745       0.248715  0.027092      0.106118


## test3:lora+pearson+mse

### 1. finetune

In [ ]:
python for_state/finetune_pipeline/scripts/run_finetune.py \
  --config for_state/finetune_pipeline/examples/lora_two_loss.yaml

### 2.preprocess infer

In [ ]:
Saving preprocessed data to /data3/fanpeishan/state/for_state/run_results/run14/c37_prep_preprocess.h5ad

### 3.infer

In [ ]:
export CUDA_VISIBLE_DEVICES=6
state tx infer \
  --model-dir /data3/fanpeishan/state/for_state/models/ST-Tahoe \
  --checkpoint /data3/fanpeishan/state/for_state/models/ST-Tahoe/final_from_preprint.ckpt \
  --pert-col 'drugname_drugconc' \
  --control-pert "[('DMSO_TF', 0.0, 'uM')]" \
  --celltype-col cell_name \
  --batch-col 'plate' \
  --adata /data3/fanpeishan/state/for_state/run_results/run14/c37_prep_preprocess.h5ad \
  --output /data3/fanpeishan/state/for_state/finetune_pipeline/results/lora_multi_loss/c37_prep_preprocess_infer.h5ad \
  --quiet \
  --lora-path /data3/fanpeishan/state/for_state/finetune_pipeline/results/lora_multi_loss/checkpoint_epoch2.pt 

### 4.eval

In [ ]:
cell-eval run \
    -ap /data3/fanpeishan/state/for_state/finetune_pipeline/results/lora_multi_loss/c37_prep_preprocess_infer.h5ad \
    -ar /data3/fanpeishan/state/for_state/run_results/run14/c37_prep.h5ad \
    -o /data3/fanpeishan/state/for_state/finetune_pipeline/results/lora_multi_loss \
    --control-pert "[('DMSO_TF', 0.0, 'uM')]" \
    --pert-col 'drugname_drugconc' \
    --profile full \
    --batch-size 1024 \
    --num-threads 10 

In [1]:
import pandas as pd
results=pd.read_csv('/data3/fanpeishan/state/for_state/finetune_pipeline/results/lora_multi_loss/agg_results.csv')
mean_results=results[results.statistic == 'mean'][['overlap_at_100','pearson_delta','mse','overlap_at_N']]
print(mean_results)

   overlap_at_100  pearson_delta       mse  overlap_at_N
2        0.083745       0.248715  0.027092      0.106118


## test4:lora+mmd_delta +mse +unbalanced sinkhorn 

### 1.finetune

### 2. infer

### 3.eval